Dealing with class imbalance:
- resampling techniques
    - under or over sampling random vs informed
    - SMOTE synthetic minor ...
- kappa statistics/ MCC Metric
- multiclass mcc "comparing two k-category assignments by a k-category correlation coeeficient"


spatial pyramid pooling in deep convolutional networks for visual recognition

In [1]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from pathlib import Path
import keras
import random
from keras.utils import Sequence
import skimage
from skimage.io import imread
from skimage.transform import resize
from skimage.util import pad
from skimage.util import crop
import numpy as np
import math

Using TensorFlow backend.


In [2]:
class MY_Gen(Sequence):

    def __init__(self, image_filenames, labels, batch_size, shuffle, train):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.num_labels = len(np.unique(labels))
        self.shuffle = shuffle
        self.on_epoch_end()
        self.train = True

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))
    
    def crop_or_pad(self, image, dim, filename):
        x, y, _ = image.shape
        if (y < dim and x < dim):
            image = pad(image, ((math.ceil((dim - image.shape[0])/2),math.floor((dim - image.shape[0])/2)),
                                (math.ceil((dim - image.shape[1])/2),math.floor((dim - image.shape[1])/2)), 
                                (0,0)), 'constant', constant_values = 255)
        elif (y >= dim and x >= dim):
            if x == 299:
                rand1 = 0
            else:
                rand1 = random.randint(1,x-dim)
            if y == 299:
                rand2=0
            else:
                rand2 = random.randint(1,y-dim)
            image = crop(image, ((rand1,x-dim-rand1),(rand2,y-dim-rand2),(0,0)))
        elif (x >= dim and y < dim):
            image = pad(image, ((0,0),
                                (math.ceil((dim - y)/2),math.floor((dim - y)/2)), 
                                (0,0)), 'constant', constant_values = 255)
            if x != 299:
                rand1 = random.randint(1,x-dim)
                image = crop(image, ((rand1,x-dim-rand1),
                                (0,0), (0,0)))
        else:
            #if y < dim: print(filename)
            image = pad(image, ((math.ceil((dim - x)/2),math.floor((dim - x)/2)),
                                (0,0), 
                                (0,0)), 'constant', constant_values = 255)
            if y != 299:
                rand2 = random.randint(1,y-dim)
                image = crop(image, ((0,0),
                                     (rand2,y-dim-rand2), (0,0)))
        return image
    
    def read_im(self, filename, dim):
        image = imread(filename)
        #image = resize(image, (dim,dim), anti_aliasing = True, mode = "reflect")
        image = skimage.color.gray2rgb(image)
        image = self.crop_or_pad(image, dim, filename)
        return image


    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        image = [self.read_im(filename, 299) for filename in batch_x]
        image = (image-np.amin(image))/(np.amax(image)-np.amin(image))
        if self.train:    
            batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
            return np.array(image), np.array(batch_y)
        else:
            return np.array(image)
        #batch_y = keras.utils.to_categorical(batch_y, self.num_labels)

    
    def on_epoch_end(self):
        if self.shuffle == True:
            fnames_and_labels = list(zip(self.image_filenames, self.labels))
            random.shuffle(fnames_and_labels)
            self.image_filenames, self.labels = zip(*fnames_and_labels)

In [3]:

path = './data/test/'
def fetch_data_set(path, ftype = 'jpg'):
    p = Path(path)
    files = list(p.glob('**/*.'+ftype))
    classes = str(files).split('/')
    classes = [classes[i] for i in list(range(2,len(classes),3)) ]
    classnames, indices = np.unique(classes, return_inverse=True)
    dict_classes = dict(zip(classnames, list(range(0,len(classes)))))
    return files, classes, dict_classes

#np.array([dict_[i] for i in classes])

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from itertools import chain

def over_under_sample(files, classes, dict_classes, num_to_undersample = 15000, num_to_oversample = 7500):
    files_array = np.array(files).reshape(-1,1)
    dic = {}
    for i in list(classes):
        dic[i] = dic.get(i,0) + 1

    classes_to_oversample = dict((k, v) for k, v in dic.items() if v < num_to_oversample)
    classes_to_undersample = dict((k, v) for k, v in dic.items() if v > num_to_undersample)    
    
    for key, value in classes_to_undersample.items():
        classes_to_undersample[key] = num_to_undersample
    for key, value in classes_to_oversample.items():
        classes_to_oversample[key] = num_to_oversample
    
    ros = RandomOverSampler(sampling_strategy = classes_to_oversample)
    rus = RandomUnderSampler(sampling_strategy = classes_to_undersample)
    x_over, y_over = ros.fit_resample(files_array, classes)
    x_under, y_under = rus.fit_resample(x_over, y_over)
    x_under = list(chain(*x_under.tolist()))
    return x_under, y_under
    
from sklearn.model_selection import train_test_split

def split_data_train_validation_test(data_X, data_Y, test_percent, validation_percent, seed): 
    assert (test_percent < 1) and (0 < validation_percent) and (validation_percent < 1)
    X_tmp, X_val, Y_tmp, Y_val = train_test_split(data_X, data_Y, test_size=validation_percent, shuffle=True, random_state=seed)
    
    if test_percent != 0:
        relative_test_percent = test_percent / (1 - validation_percent)
        X_train, X_test, Y_train, Y_test = train_test_split(X_tmp, Y_tmp, test_size=relative_test_percent, shuffle=True, random_state=seed)
        split_data = [X_train, Y_train, X_val, Y_val, X_test, Y_test]
    else:
        X_train, Y_train = X_tmp, Y_tmp
        split_data = [X_train, Y_train, X_val, Y_val]

    return split_data  

def encode_labels(labels, OneHot=True, encoder = None):
    if OneHot and encoder == None:
        from sklearn.preprocessing import OneHotEncoder
        enc = OneHotEncoder()
        enc.fit(np.array(labels).reshape(-1, 1))
        OneHot = enc.transform(np.array(labels).reshape(-1, 1)).toarray()
        return OneHot, enc
    else:
        OneHot = encoder.transform(np.array(labels).reshape(-1, 1)).toarray()
        return OneHot
    # mlb 
    #1 convert labels to multilabel using hardcoded dict
    #2 fit mlb
    #3 transform 
    # return labels and encoder
    # repeat for when encoder is present
 


In [4]:
# read dataset
files, classes, dict_classes = fetch_data_set("./data/imgs/")
# split dataset
split = split_data_train_validation_test(files, classes, 0.05, .25, random.randint(1,10000))
split[0], split[1] = over_under_sample(split[0], split[1], dict_classes, num_to_undersample = 3000, num_to_oversample = 3000)
# load label encoder and transform labels
split[1], OH_enc = encode_labels(split[1], OneHot = True)
split[3] = encode_labels(split[3], encoder = OH_enc)





#from collections import Counter
#print(sorted(Counter(classes).items()))
#print(sorted(Counter(y_re).items()))

In [ ]:
#c,_= np.unique(split[5], return_inverse=True)
#len(c)
#b = {}
#for i in list(split[5]):
#    b[i] = b.get(i,0) + 1

#counts = dict((k, v) for k, v in b.items() if v > 1)
#counts

#f, c, d = fetch_data_set('./data/imgs')
#d['cavo'] = [0,1,1]
#[d[i] for i in c]
d_ = d
d_['Annelida'] = ['Annelida', 'Metazoa', 'Eukaryota']
d_['Bivalvia__Molusca'] = ['Bivalvia__Molusca', 'Mollusca', 'Metazoa', 'Eukaryota']
d_['Brachyura'] = ['Brachyura', 'Decapoda', 'Malacostraca', 'Arthropoda', 'Metazoa', 'Eukaryota']
d_['Candaciidae'] = ['Candaciidae', 'Calanoida', 'Copepoda', 'Maxillopoda', 'Arthropoda', 'Metazoa', 'Eukaryota']


In [ ]:
p = Path('./data/test/') 
files = list(p.glob('**/*.jpg'))
classes = str(files).split('/')
classes = [classes[i] for i in list(range(2,len(classes),3)) ]
classnames, indices = np.unique(classes, return_inverse=True)
labels = keras.utils.to_categorical(indices, len(np.unique(indices)))

In [ ]:
p = Path('./data/imgs/') 
files = list(p.glob('**/*.jpg'))
classes = str(files).split('/')
classes = [classes[i] for i in list(range(2,len(classes),3)) ]
classnames, indices = np.unique(classes, return_inverse=True)

In [ ]:
len(split[1])

In [ ]:
np.std(list(counts.values()))
counts
classes_to_oversample = dict((k, v) for k, v in b.items() if v < 7500)
classes_to_undersample = dict((k, v) for k, v in b.items() if v > 15000)
#assign new target values for sampling
for key, value in classes_to_undersample.items():
    classes_to_undersample[key] = 15000
classes_to_undersample.values()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
d = {0: 150, 1: 300}
ros = RandomOverSampler(random_state = 0, sampling_strategy = d)
x_re, y_re = ros.fit_resample(np.array(split[0]).reshape(-1,1), split[1])
from collections import Counter
print(sorted(Counter(split[1]).items()))
print(sorted(Counter(y_re).items()))

In [ ]:
from itertools import chain

x_re = list(chain(*x_re.tolist()))
y_re


In [ ]:
#import keras_metrics
#metrics=[keras_metrics.precision(), keras_metrics.recall()
b = {}
for i in list(np.array(x_re)):
    b[i] = b.get(i,0) + 1

dict((k, v) for k, v in b.items() if v > 1)

#x_re

In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import keras_metrics
from keras import backend as K

base_model = InceptionV3(weights='imagenet', include_top=False)


x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)

predictions = Dense(40, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
batch_size=30
#num_training_samples=len(files)

files, classes, dict_classes =  fetch_data_set('./data/test')
split = split_data_train_validation_test(files, classes, .1, .25, 666)

In [ ]:
pd.crosstab(np.array([dict_classes[i] for i in split[5]]), (test_pred), rownames=['Actual'], colnames=['Predicted'], margins=True)
#len(split[5]), len(test_pred)
#len(np.argmax(model_pred, axis = 1, out = None)), len(test_pred), len(split[5])
#[dict_classes[i] for i in split[5]], list(test_pred)

In [ ]:
#[dict_classes[i] for i in split[1]]
import pandas as pd
model_pred = model.predict_generator(generator = test_batch, steps = (len(split[4]) // batch_size))
#test_actu = np.argmax([dict_classes[i] for i in split[5]], axis = 1, out = None)
test_pred = np.argmax(model_pred, axis = 1, out = None)
pd.crosstab(split[5], test_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)


In [6]:
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint

def step_decay_schedule(base_lr=1e-4, decay_factor=0.5, step_decay=5):
    def schedule(epoch):
        ## Multiply learning rate by 'decay_factor' every 'step_size' epochs (note that epoch is indexed from 0):
        updated_lr = base_lr * (decay_factor ** np.floor((epoch + 1) / step_decay))  
        return updated_lr    
    return LearningRateScheduler(schedule)

batch_size = 30
training_batch = MY_Gen(split[0], split[1], batch_size, shuffle = True, train = True)
validation_batch = MY_Gen(split[2], split[3], batch_size, shuffle = True, train = True)
test_batch = MY_Gen(split[4], split[5], batch_size, shuffle = False, train = False)

lr_policy = step_decay_schedule(base_lr=0.01, decay_factor=0.75, step_decay=10)
callback_list = [lr_policy]
callback_list.append(EarlyStopping(monitor='val_loss',
                                   min_delta=0,
                                   patience=0,
                                   verbose=0,
                                   mode='auto',
                                   baseline=None,
                                   restore_best_weights=False))
callback_list.append(ModelCheckpoint("./checkpoints/model_{epoch:02d}.hdf5", 
                                              monitor='val_loss', 
                                              verbose=0, 
                                              save_best_only=True, save_weights_only=False))
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=[keras_metrics.precision(), keras_metrics.recall(),'accuracy'])

history = model.fit_generator(generator=training_batch,
                    validation_data = validation_batch,
                    validation_steps = (len(split[2]) // batch_size),
                    steps_per_epoch=(len(split[0]) // batch_size),
                    epochs=5,
                    verbose=1,
                    callbacks = callback_list,
                    use_multiprocessing=True,
                    workers=16,
                    max_queue_size=32)

Epoch 1/5
 267/4000 [=>............................] - ETA: 27:53 - loss: 15.7199 - precision: 0.0000e+00 - recall: 0.0000e+00 - acc: 0.0215

Process ForkPoolWorker-20:
Process ForkPoolWorker-12:
Process ForkPoolWorker-26:
Process ForkPoolWorker-21:
Process ForkPoolWorker-19:
Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-7:
Process ForkPoolWorker-9:
Process ForkPoolWorker-6:
Process ForkPoolWorker-16:
Process ForkPoolWorker-3:
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-27:
Process ForkPoolWorker-8:
Process ForkPoolWorker-24:
Process ForkPoolWorker-25:
Process ForkPoolWorker-23:
Process ForkPoolWorker-22:
Process ForkPoolWorker-18:
Process ForkPoolWorker-4:
Process ForkPoolWorker-17:
Process ForkPoolWorker-10:
Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Process ForkPoolWorker-31:
Process ForkPoolWorker-29:
Process ForkPoolWorker-32:
Process ForkPoolWorker-28:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/

  File "<ipython-input-2-0e3df50a5473>", line 59, in __getitem__
    image = [self.read_im(filename, 299) for filename in batch_x]
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/lib/python3.5/dist-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", 

KeyboardInterrupt: 

In [ ]:
def plot_cm_calib(model, data, labels, OH_enc, dict_classes, batch_size = 100):
    import pandas as pd
    #split[5] = encode_labels(split[5], OneHot=True, encoder = OH_enc)
    test_batch = MY_Gen(data, labels, batch_size = batch_size, shuffle = False, train = False)
    model_pred = model.predict_generator(generator = test_batch, steps = (len(data) // batch_size))
    t = {v: k for k, v in dict_classes.items()}
#test_actu = OH_enc.inverse_transform(split[5]).flatten()
    test_pred = [t[i] for i in np.argmax(model_pred, axis = 1, out = None)]
    cm = pd.crosstab(np.array(labels), np.array(test_pred), rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    bins = np.arange(0,1.1,0.1)
    acc = np.zeros([10,1])
    pred = model_pred
    actu = np.array(test_pred) == np.array(labels)
    actu = actu.reshape([len(data),1])
    for i in range(len(bins)-1):
        pred = pred*(OH_enc.transform(np.array(labels).reshape(-1,1)).toarray() == 1)
        tmp = ((pred > bins[i]) & (pred < bins[i+1]))
        if sum(tmp.flatten()) == 0:
            acc[i] = 0
        else:
            acc[i] = np.sum(tmp)/np.sum(pred > 0)
    plt.figure(figsize=(10,5))
    plt.bar(np.arange(0,10,1), height = acc.flatten())
    plt.xticks(np.arange(0,10,1), ['0-0.1','0.1-0.2','0.2-0.3','0.3-0.4','0.4-0.5','0.5-0.6','0.6-0.7','0.7-0.8','0.8-0.9','0.9-1'])
    plt.xlabel("Softmax Intervals")
    plt.ylabel("Accuracy")
    plt.title("Sofmax calibration")
    plt.show()
    plt.figure(figsize=(10,5))
    plt.hist(pred.flatten(), range = (0,1))
    plt.xlabel("Softmax Intervals")
    plt.ylabel("No of Samples")
    plt.title("Sofmax calibration")
    plt.show()    
    return cm
plot_cm_calib(model, split[2], split[3], OH_enc, dict_classes, batch_size = 75)

Your classifier probably misclassifies some images. Investigate how well the accuracy of the
classifications match the softmax output values, for instance as a histogram with softmax
intervals (buckets) on the x-axis (e.g. 0.70-0.75), and the percentage correct classifications for
each bucket (e.g. 68%) on the y-axis. Is your classifier overconfident or underconfident, or
neutral?

In [ ]:
json_string = model.to_json()
with open("data/output/models/pretra_INC.json", "w") as json_file:
    json_file.write(json_string)

model.save_weights("data/output/models/pretra_INC_weights.hdf5")

In [ ]:
#im=np.asarray(Image.open(files[1]).resize([299,299]))
#im = im/np.amax(im)
#import matplotlib.pyplot as plt
#plt.imshow(image[7])
#plt.show()
#ind = np.arange(105)
#isinstance(classes, list)
#len(classes)
import random
c=list(zip(files,classes))
random.shuffle(c)
files,classes = zip (*c)

In [ ]:
import skimage
im = resize(imread(files[1]), (100, 100))
im = skimage.color.gray2rgb(im)
im.shape

In [ ]:
#im = np.array([(Image.open(file_name).resize([299,299])) for file_name in files])
#im2 = np.array([
           # resize(imread(file_name), (299, 299))
            #   for file_name in files])
#len(im2)
#im2.shape
#im.shape
def norm_im(filename, dim):
    image = imread(filename)
    image = resize(image, (dim,dim), mode = "edge")
    image = (image-np.amin(image))/(np.amax(image)-np.amin(image))
    return image
image = np.array([norm_im(filename, 100) for filename in files])


image.shape

In [ ]:
#files[range(1,10)]
list(indices)
keras.utils.to_categorical(indices, 5)
len(np.unique(indices))

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')




In [ ]:
batch_size=100
num_training_samples=len(files)
# compile the model (should be done *after* setting layers to non-trainable)
my_training_batch_generator = MY_Gen(files, labels, batch_size)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])
# train the model on the new data for a few epochs
model.fit_generator(generator=my_training_batch_generator,
                                          steps_per_epoch=(num_training_samples // batch_size),
                                          epochs=10,
                                          verbose=1,
                                          use_multiprocessing=False,
                                          workers=16,
                                          max_queue_size=32,
                             shuffle = True)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.



In [ ]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)

In [ ]:
from pathlib import Path
p = Path('./data/imgs/') 
classes = [x for x in p.iterdir() if x.is_dir()]
files = list(p.glob('**/*.jpg'))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
labels = [
    ("legs","blue", "jeans"),
    ("no_legs","blue", "dress"),
    ("no_legs","red", "dress"),
    ("no_legs","red", "shirt"),
    ("no_legs","blue", "shirt"),
    ("no_legs","black", "jeans")]
mlb = MultiLabelBinarizer()
mlb.fit(labels)

mlb.classes_

mlb.transform([("blue", "dress", "no_legs")]), mlb.inverse_transform(np.array([[0, 1, 1, 0, 0, 1, 0, 0], [1, 1, 1, 0, 0, 1, 0, 0]]))
#np.array()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(np.array(split[5]).reshape(-1, 1))
OneHot=enc.transform(np.array(split[5]).reshape(-1, 1)).toarray()
#enc.inverse_transform(OneHot)
OneHot[1:5]

In [ ]:
p = Path('./data/test')
end = '.jpg'
list(p.glob('**/*'+end))
